In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import sqlite3
import pandas as pd

In [10]:
conn = sqlite3.connect('bank.db')
customer_pandas_df = pd.read_sql_query('SELECT * FROM customer', conn)
transaction_pandas_df = pd.read_sql_query('SELECT * FROM "transaction"', conn)

In [11]:
transaction_pandas_df.head(4)

,id,branchid,custid,type,date,amt
0,1,3,26,wth,1629068862,9800
1,2,3,2,wth,1629068862,1100
2,3,3,17,dep,1629068862,4300
3,4,3,5,wth,1629068862,4100


In [12]:
spark = SparkSession.builder.appName('test').getOrCreate()

## Spark EDA

- richest customer

- poorest customer

- richest customer’s favorite branch

- poorest customer’s favorite branch

- net total of each branch

In [13]:
customer_spark_df = spark.createDataFrame(customer_pandas_df)

In [14]:
transaction_spark_df = spark.createDataFrame(transaction_pandas_df)

### richest customer

In [15]:
richest_customer = customer_spark_df.sort('balance', ascending=False)

In [16]:
richest_customer.show(1)

+------+----------+----------+-----+-------+
|custid|createdate|     fname|lname|balance|
+------+----------+----------+-----+-------+
|    13|1629076179|Kristopher|Kelly|77100.0|
+------+----------+----------+-----+-------+
only showing top 1 row



In [17]:
richest_customer_id = richest_customer.select('custid')

In [18]:
richest_customer_id = richest_customer_id.head()[0]

### poorest customer

In [19]:
poorest_customer = customer_spark_df.sort('balance')

In [20]:
poorest_customer_id = poorest_customer.select('custid')

In [21]:
poorest_customer_id = poorest_customer.head()[0]

In [22]:
customer_spark_df.sort('balance').show(1)

+------+----------+-----+------+---------+
|custid|createdate|fname| lname|  balance|
+------+----------+-----+------+---------+
|     7|1629076177| Sean|Burton|-245700.0|
+------+----------+-----+------+---------+
only showing top 1 row



### richest customer’s favorite branch

In [23]:
customers_and_transactions = customer_spark_df.join(transaction_spark_df, on='custid')

In [24]:
richest_cust_transactions = customers_and_transactions.filter(customers_and_transactions['custid'] == richest_customer_id)

In [25]:
richest_cust_transactions.groupBy('custid', 'branchid').count().orderBy('count', ascending=False).show(1)

+------+--------+-----+
|custid|branchid|count|
+------+--------+-----+
|    13|       3|  110|
+------+--------+-----+
only showing top 1 row



### poorest customer’s favorite branch

In [26]:
poorest_cust_transactions = customers_and_transactions.filter(customers_and_transactions['custid'] == poorest_customer_id)

In [27]:
poorest_cust_transactions.groupBy('custid', 'branchid').count().orderBy('count', ascending=False).show(1)

+------+--------+-----+
|custid|branchid|count|
+------+--------+-----+
|     7|       3|  109|
+------+--------+-----+
only showing top 1 row



### net total of each branch

In [28]:
transaction_spark_df.groupBy('branchid').sum('amt').show()

+--------+--------+
|branchid|sum(amt)|
+--------+--------+
|       1| 1819800|
|       3| 1483600|
|       2| -460000|
+--------+--------+

